In [1]:
import sys
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, GaussianNoise, LSTM, Bidirectional, Dropout, Dense, Conv1D
from tensorflow.keras import regularizers, backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.metrics import RootMeanSquaredError, R2Score

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.TreeTimeML_save import TreeTimeML

import pandas as pd
import numpy as np
import polars as pl
import datetime
import seaborn as sns
import lightgbm as lgb
import random
import matplotlib.pyplot as plt
import logging
import time
import re

In [2]:
params = {
    "daysAfterPrediction": 5,
    'timesteps': 25,
    'target_option': 'last',
    "TreeTime_isFiltered": True,
    "TreeTime_volatility_atr_qup": 0.90,
    "TreeTime_top_highest": 10,
    
    "TreeTime_lstm_units": 64,
    "TreeTime_lstm_num_layers": 3,
    "TreeTime_lstm_dropout": 0.00001,
    "TreeTime_lstm_recurrent_dropout": 0.00001,
    "TreeTime_lstm_learning_rate": 0.001,
    "TreeTime_lstm_optimizer": "adam",
    "TreeTime_lstm_bidirectional": True,
    "TreeTime_lstm_batch_size": 2**14,
    "TreeTime_lstm_epochs": 4,
    "TreeTime_lstm_l1": 0.00001,
    "TreeTime_lstm_l2": 0.00001,
    "TreeTime_inter_dropout": 0.00001,
    "TreeTime_input_gaussian_noise": 0.00001,
    "TreeTime_lstm_conv1d": True,
    "TreeTime_lstm_conv1d_kernel_size": 3,
    "TreeTime_lstm_loss": "mse",
}

In [3]:
stock_group = "group_snp500_finanTo2011"

start_train_date = datetime.date(year=2014, month=1, day=1)

eval_dates = [
    datetime.date(2015,  3, 15),
    datetime.date(2016,  6, 10),
    datetime.date(2017,  9,  5),
    datetime.date(2018, 12, 20),
    datetime.date(2019,  2, 13),
    datetime.date(2020,  5, 25),
    datetime.date(2021,  8, 30),
    datetime.date(2022, 11,  7),
    datetime.date(2023,  1, 14),
    datetime.date(2024,  7,  4),
    datetime.date(2025,  2,  1),
]

treetimeML_instances = [
    TreeTimeML(
        train_start_date=start_train_date,
        test_dates=[eval_date],
        group=stock_group,
        params=params
    )
    for eval_date in eval_dates
]

In [4]:
for tm in treetimeML_instances:
    tm.load_and_filter_sets(main_path="../src/featureAlchemy/bin/")

Test date 2015-03-15 not found in the database. Resetting to last trading day.
Test date 2020-05-25 not found in the database. Resetting to last trading day.
Test date 2023-01-14 not found in the database. Resetting to last trading day.
Test date 2024-07-04 not found in the database. Resetting to last trading day.
Test date 2025-02-01 not found in the database. Resetting to last trading day.


In [8]:
# Collect lists of the desired attributes
train_Xtree_list = [tm.train_Xtree   for tm in treetimeML_instances]
train_ytree_list = [tm.train_ytree   for tm in treetimeML_instances]
train_Xtime_list = [tm.train_Xtime   for tm in treetimeML_instances]
train_ytime_list = [tm.train_ytime   for tm in treetimeML_instances]

test_Xtree_list  = [tm.test_Xtree    for tm in treetimeML_instances]
test_ytree_list  = [tm.test_ytree    for tm in treetimeML_instances]
test_Xtime_list  = [tm.test_Xtime    for tm in treetimeML_instances]
test_ytime_list  = [tm.test_ytime    for tm in treetimeML_instances]

treenames_list   = [tm.featureTreeNames for tm in treetimeML_instances]
timenames_list   = [tm.featureTimeNames for tm in treetimeML_instances]

In [9]:
sep = ["Category", "Seasonal", "lag", "FeatureGroup"]
indices = [i for i, s in enumerate(treenames_list[0])
           if not any(sub in s for sub in sep)]
print(indices)  
print(len(indices))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 119

In [7]:
from typing import Optional
import numpy as np
import torch
from numpy.typing import NDArray
#from tqdm import tqdm
def optimize_threshold_sigmoid(
    A: NDArray[np.float32],
    v: NDArray[np.float32],
    ao_idx: int,
    q: float,
    lr: float = 1e-2,
    epochs: int = 200,
    sharpness: float = 5.0,
    probs_noise_std: float = 0.5,
    subsample_ratio: float = 0.1,
    maximize: bool = True,
    upper_quantile: bool = True,
    device: Optional[str] = None
) -> NDArray[np.float32]:
    # Determine device
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    
    extreme_sign = -1.0 if maximize else 1.0
    upper_sign = 1.0 if upper_quantile else -1.0

    # Convert inputs to torch tensors
    A_tensor = torch.tensor(A, dtype=torch.float32, device=device)  # (N, D)
    v_tensor = torch.tensor(v, dtype=torch.float32, device=device)  # (N,)

    # Initialize a
    D = A_tensor.shape[1]
    a = torch.nn.Parameter(torch.zeros(D, dtype=torch.float32, device=device) * 0.25)
    a.data[ao_idx] = 1.0

    # Optimizer
    optimizer = torch.optim.Adam([a], lr=lr)

    # Training loop
    subsample_size = int(A.shape[0] * subsample_ratio)
    #pbar = tqdm(range(epochs), desc="Epochs")
    #for epoch in pbar:
    for epoch in range(epochs):
        optimizer.zero_grad()
        w = A_tensor.matmul(a)               # (N,)
        thresh = torch.quantile(w, q)        # scalar
        probs = torch.sigmoid(upper_sign * sharpness * (w - thresh))
        # --- regularisation: noise + subsample ---
        probs = probs + torch.randn_like(probs) * probs_noise_std
        probs = probs.clamp(0,1)
        idx = torch.randperm(probs.size(0), device=device)[:subsample_size]
        probs_sub, v_sub = probs[idx], v_tensor[idx]
        loss = extreme_sign * (probs_sub * v_sub).sum() / (probs_sub.sum() + 1e-8)
        loss.backward()
        optimizer.step()
        
        #mean_v = (probs_sub * v_sub).sum() / (probs_sub.sum() + 1e-8)
        #pbar.set_postfix(mean_v=f"{mean_v:.4f}")
        
    # Return optimized `a` as numpy array
    return a.detach().cpu().numpy()


In [115]:
A = train_Xtree[:, indices]  # Select relevant features
ao_idx = list(treenames[indices]).index("FeatureTA_volatility_atr")
q = 0.90

splitratio = 0.90
n_total = train_Xtree.shape[0]
split_at = int(n_total * splitratio)
A_train, A_holdout = A[:split_at], A[split_at:]
y_train, y_holdout = train_ytree[:split_at], train_ytree[split_at:]

# Define base args and variant overrides for DRY
base_ot_params = dict(
    A=A_train,
    v=y_train,
    ao_idx=ao_idx,
    q=q,
    lr=1e-2,
    epochs=200,
    sharpness=5.0,
    probs_noise_std=0.5,
    subsample_ratio=0.1,
)
variant_params = {
    "up_max": {"maximize": True, "upper_quantile": True, "q": q},
    "up_min": {"maximize": False, "upper_quantile": True, "q": q},
    "do_max": {"maximize": True, "upper_quantile": False, "q": 1-q},
    "do_min": {"maximize": False, "upper_quantile": False, "q": 1-q},
}
a_results = {}
for name, overrides in variant_params.items():
    args = {**base_ot_params, **overrides}
    a_results[name] = optimize_threshold_sigmoid(**args)
    time.sleep(0.1)  # Small delay to avoid overwhelming the system

# Unpack results
a_np_up_max = a_results["up_max"]
a_np_up_min = a_results["up_min"]
a_np_do_max = a_results["do_max"]
a_np_do_min = a_results["do_min"]

Epochs: 100%|██████████| 200/200 [00:02<00:00, 82.68it/s, mean_v=1.0097]

Epochs: 100%|██████████| 200/200 [00:02<00:00, 89.24it/s, mean_v=1.0024]

Epochs: 100%|██████████| 200/200 [00:02<00:00, 88.94it/s, mean_v=1.0104]

Epochs: 100%|██████████| 200/200 [00:02<00:00, 89.14it/s, mean_v=1.0007]



In [118]:
def ideal_up_mean(A, a, v, q):
    w = A @ a
    threshold = np.quantile(w, q)
    mask = w > threshold
    print(np.mean(v[mask]))
def ideal_do_mean(A, a, v, q):
    w = A @ a
    threshold = np.quantile(w, q)
    mask = w < threshold
    print(np.mean(v[mask]))

print("Ideal Up Max:")
ideal_up_mean(A_train, a_np_up_max, y_train, q)
ideal_up_mean(A_holdout, a_np_up_max, y_holdout, q)

print("Ideal Up Min:")
ideal_up_mean(A_train, a_np_up_min, y_train, q)
ideal_up_mean(A_holdout, a_np_up_min, y_holdout, q)

print("Ideal Down Max:")
ideal_do_mean(A_train, a_np_do_max, y_train, 1-q)
ideal_do_mean(A_holdout, a_np_do_max, y_holdout, 1-q)

print("Ideal Down Min:")
ideal_do_mean(A_train, a_np_do_min, y_train, 1-q)
ideal_do_mean(A_holdout, a_np_do_min, y_holdout, 1-q)

a0 = np.zeros(A.shape[1])  # Initial guess
a0[ao_idx] = 1.0
ideal_up_mean(A_train, a0, y_train, q)
ideal_up_mean(A_holdout, a0, y_holdout, q)

Ideal Up Max:
1.02881
1.0215393
Ideal Up Min:
0.9867375
1.0100167
Ideal Down Max:
1.0279098
1.0141095
Ideal Down Min:
0.98584956
1.0028721
1.0086963
1.0200106


In [113]:
w = A_holdout @ a_np_up_max
threshold = np.quantile(w, q)
mask = w > threshold

d = np.where(mask)[0]
ddiff = np.diff(np.sort(d))
print(max(ddiff))

459


## Subtract positive from negative

In [122]:
def subtract_mean(A, a_max, a_min, v, q, up_max=True, up_min=False):
    w_max = A @ a_max
    w_min = A @ a_min
    threshold_max = np.quantile(w_max, q)
    threshold_min = np.quantile(w_min, q)
    mask_max = w_max > threshold_max if up_max else w_max < threshold_max
    mask_min = w_min > threshold_min if up_min else w_min < threshold_min
    # in mask_max but not in max_min
    mask = mask_max & ~mask_min
    print(f"Mean over mask {np.mean(v[mask])}")
    print(f"Mean over mask_min {np.mean(v[mask_min])}")
    print(f"Mean over mask_max {np.mean(v[mask_max])}")
    print(f"Sum both masks {(mask_max & mask_min).sum()/ mask_max.sum()}")
    return mask
    
mask = subtract_mean(A_train, a_np_up_max, a_np_do_min, y_train, q, up_max=True, up_min=False)
subtract_mean(A_holdout, a_np_up_max, a_np_do_min, y_holdout, q, up_max=True, up_min=False)

d = np.where(mask)[0]
ddiff = np.diff(np.sort(d))
print(max(ddiff))

Mean over mask 1.0288593769073486
Mean over mask_min 1.0044536590576172
Mean over mask_max 1.0288100242614746
Sum both masks 0.8181818181818182
Mean over mask 1.037727952003479
Mean over mask_min 1.0103803873062134
Mean over mask_max 1.021539330482483
Sum both masks 0.7648902821316614
1625


## Optuna Trials

In [12]:
import numpy as np
import optuna

# Parameters
sep = ["Category", "Seasonal", "lag", "FeatureGroup"]
indices = [i for i, s in enumerate(treenames_list[0]) if not any(sub in s for sub in sep)]
ao_idx = list(treenames_list[0][indices]).index("FeatureTA_volatility_atr")
q = 0.90
splitratio = 0.80

# Prepare lists of train/holdout splits across all instances
A_train_list, A_holdout_list = [], []
y_train_list, y_holdout_list = [], []

for tm in treetimeML_instances:
    A = tm.train_Xtree[:, indices]
    n_total = A.shape[0]
    split_at = int(n_total * splitratio)
    A_train_list.append(A[:split_at])
    A_holdout_list.append(A[split_at:])
    y_train_list.append(tm.train_ytree[:split_at])
    y_holdout_list.append(tm.train_ytree[split_at:])

# Base Optuna datasets definitions (can include more variants)
datasets = {
    "up_max": dict(maximize=True,  upper_quantile=True,  q=q),
    "up_min": dict(maximize=False, upper_quantile=True,  q=q),
    "do_max": dict(maximize=True,  upper_quantile=False, q=1-q),
    "do_min": dict(maximize=False, upper_quantile=False, q=1-q),
}

def objective(trial, variant):
    # sample hyper-parameters
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    epochs = trial.suggest_int("epochs", 50, 1800)
    sharpness = trial.suggest_float("sharpness", 0.05, 50.0, log=True)
    probs_noise_std = trial.suggest_float("probs_noise_std", 0.0, 1.0)
    subsample_ratio = trial.suggest_float("subsample_ratio", 0.01, 1.0)

    scores = []
    for A_tr, A_hd, y_tr, y_hd in zip(A_train_list, A_holdout_list, y_train_list, y_holdout_list):
        params = {
            "A": A_tr,
            "v": y_tr,
            "ao_idx": ao_idx,
            "q": datasets[variant]["q"],
            "maximize": datasets[variant]["maximize"],
            "upper_quantile": datasets[variant]["upper_quantile"],
            "lr": lr,
            "epochs": epochs,
            "sharpness": sharpness,
            "probs_noise_std": probs_noise_std,
            "subsample_ratio": subsample_ratio,
        }
        a = optimize_threshold_sigmoid(**params)
        w = A_hd @ a
        th = np.quantile(w, params["q"])
        mask = w > th if params["upper_quantile"] else w < th
        scores.append(np.mean(y_hd[mask]))
        
    print(f"Variant: {variant}, Scores: {np.round(scores, 3)}, Mean Score: {np.mean(scores):.4f}")
    return np.mean(np.log(scores))

# Run one study per variant, optimizing the mean holdout score across all instances
timeout = 60*60
best = {}
for name in datasets:
    direction = "maximize" if datasets[name]["maximize"] else "minimize"
    study = optuna.create_study(direction=direction, study_name=name)
    study.optimize(lambda t: objective(t, name), timeout=timeout)
    best[name] = study.best_params
    
    df = study.trials_dataframe()
    print("\nTrials DataFrame:")
    print(df.sort_values("value").to_string())
    param_importances = optuna.importance.get_param_importances(study)
    print("Parameter Importances:")
    for key, value in param_importances.items():
        print(f"{key}: {value}")


[I 2025-07-02 18:47:18,973] A new study created in memory with name: up_max
[I 2025-07-02 18:47:39,472] Trial 0 finished with value: 0.015307474881410599 and parameters: {'lr': 0.0436302310540298, 'epochs': 512, 'sharpness': 7.08556004692333, 'probs_noise_std': 0.779363398155941, 'subsample_ratio': 0.247428419547648}. Best is trial 0 with value: 0.015307474881410599.


Variant: up_max, Scores: [1.03  1.027 1.004 1.013 1.009 1.036 1.008 1.014 1.008 1.009 1.012], Mean Score: 1.0155


[I 2025-07-02 18:48:30,782] Trial 1 finished with value: 0.019121911376714706 and parameters: {'lr': 0.039618888941629606, 'epochs': 1317, 'sharpness': 0.5456951290850517, 'probs_noise_std': 0.05676341263428486, 'subsample_ratio': 0.9678934172422053}. Best is trial 1 with value: 0.019121911376714706.


Variant: up_max, Scores: [1.027 1.025 1.006 1.01  1.017 1.051 1.037 1.012 1.008 1.008 1.013], Mean Score: 1.0194


[I 2025-07-02 18:48:42,749] Trial 2 finished with value: 0.01267212349921465 and parameters: {'lr': 0.013378914366535991, 'epochs': 294, 'sharpness': 5.664302243226896, 'probs_noise_std': 0.823548164969008, 'subsample_ratio': 0.05461834041425086}. Best is trial 1 with value: 0.019121911376714706.


...

Variant: do_min, Scores: [1.019 1.016 1.006 1.    0.998 1.014 1.017 1.007 1.008 1.004 1.004], Mean Score: 1.0083


[I 2025-07-02 22:47:48,367] Trial 118 finished with value: 0.006741927471011877 and parameters: {'lr': 0.00024165135851181028, 'epochs': 474, 'sharpness': 31.172009893121416, 'probs_noise_std': 0.5898489558110015, 'subsample_ratio': 0.35553424921722826}. Best is trial 110 with value: 0.005910201463848352.


Variant: do_min, Scores: [1.013 1.016 1.004 1.003 1.003 1.002 1.013 1.006 1.008 1.004 1.002], Mean Score: 1.0068


[I 2025-07-02 22:48:06,309] Trial 119 finished with value: 0.00800253078341484 and parameters: {'lr': 0.00012142705800928953, 'epochs': 499, 'sharpness': 31.00405725257006, 'probs_noise_std': 0.6611580497047739, 'subsample_ratio': 0.35923130053086955}. Best is trial 110 with value: 0.005910201463848352.


Variant: do_min, Scores: [1.017 1.021 1.003 1.005 1.006 1.004 1.009 1.008 1.008 1.004 1.004], Mean Score: 1.0081

Trials DataFrame:
     number     value             datetime_start          datetime_complete               duration  params_epochs  params_lr  params_probs_noise_std  params_sharpness  params_subsample_ratio     state
110     110  0.005910 2025-07-02 22:45:24.840312 2025-07-02 22:45:38.686590 0 days 00:00:13.846278            384   0.000245                0.542199         35.812613                0.412722  COMPLETE
108     108  0.005929 2025-07-02 22:44:54.758433 2025-07-02 22:45:09.591333 0 days 00:00:14.832900            417   0.000236                0.478025         33.672546                0.306379  COMPLETE
115     115  0.006106 2025-07-02 22:46:37.748260 2025-07-02 22:46:57.107254 0 days 00:00:19.358994            538   0.000300                0.625043         30.870438                0.417029  COMPLETE
107     107  0.006219 2025-07-02 22:44:41.278915 2025-07-02 22:4